# Importando as bibliotecas necessárias
Importando as bibliotecas necessárias, incluindo pandas, matplotlib, seaborn e numpy.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid")

# Carregar o Dataset

O trecho de código realiza uma análise exploratória inicial no dataset `listagem-alunos.csv` usando a biblioteca Pandas. As seguintes etapas são executadas:

1. **Carregamento do Dataset:**  
   - O arquivo CSV é lido e armazenado no DataFrame `df`.

2. **Dimensão do Dataset:**  
   - `df.shape` retorna o número total de linhas (registros) e colunas (variáveis), exibindo essa informação no console.

3. **Verificação de Valores Nulos:**  
   - `df.isnull().sum()` contabiliza a quantidade de valores nulos por coluna, auxiliando na identificação de dados ausentes.

4. **Tipos de Dados:**  
   - `df.dtypes` exibe os tipos de dados de cada coluna, facilitando a análise e possíveis conversões de tipos.

5. **Visualização das Primeiras Linhas:**  
   - `df.head()` apresenta as cinco primeiras linhas do dataset para uma inspeção inicial dos dados.

In [ ]:
# Load CSV Data
df = pd.read_csv('listagem-alunos.csv',sep=';', on_bad_lines='skip')

num_linhas, num_colunas = df.shape
print(f"Número total de registros (linhas): {num_linhas}")
print(f"Número total de variáveis (colunas): {num_colunas}")
print("\n")

print("Valores nulos:")
print(df.isnull().sum())
print("\n")

print("Tipos de dados:")
print(df.dtypes)
print("\n")

df.head()

# Remoção dos Campos Desnecessários

Nesta etapa, vamos remover os campos (colunas) que não são relevantes para nossa análise dos dados de alunos. Geralmente, isso é necessário para:

- Simplificar o conjunto de dados
- Melhorar a performance dos processos de análise
- Eliminar informações redundantes ou irrelevantes

In [ ]:
colunas_remover = ['CO_IES', 'NO_IES', 'SG_IES', 'CO_INSCRICAO_ENEN','NO_INSCRICAO', 'NO_MODALIDADE_CONCORRENCIA','ST_BONUS_PERC','QT_BONUS_PERC','NO_ACAO_AFIRMATIVA_BONUS']  

df = df.drop(columns=colunas_remover, errors='ignore')

df['NU_NOTA_CANDIDATO'] = pd.to_numeric(df['NU_NOTA_CANDIDATO'], errors='coerce')
df['NU_NOTACORTE_CONCORRIDA'] = pd.to_numeric(df['NU_NOTACORTE_CONCORRIDA'], errors='coerce')
df['NU_CLASSIFICACAO'] = pd.to_numeric(df['NU_CLASSIFICACAO'], errors='coerce')

df.dropna(subset=['NU_NOTA_CANDIDATO', 'NU_NOTACORTE_CONCORRIDA','NU_CLASSIFICACAO'], inplace=True)

print("Tipos de dados:")
print(df.dtypes)
print("\n")

df.head()

# 1.Análise de Competitividade por Curso e Instituição

## Nota de Corte vs. Nota do Candidato
Verificar se há cursos onde a nota média dos candidatos admitidos (NU_NOTA_CANDIDATO) é significativamente superior à nota de corte, indicando alta competitividade.

1. **Agrupamento e Cálculo da Média:**
   - O código agrupa o DataFrame pelo campo `NO_CURSO` e calcula a média das notas dos candidatos (`NU_NOTA_CANDIDATO`) para cada curso.

2. **Obtenção da Nota de Corte:**
   - Para cada curso, é obtida a primeira ocorrência da nota de corte (`NU_NOTACORTE_CONCORRIDA`) usando o agrupamento por `NO_CURSO`.

3. **Cálculo da Diferença:**
   - É calculada a diferença entre a média das notas dos candidatos e a nota de corte para cada curso.

4. **Seleção dos Cursos Altamente Competitivos:**
   - São selecionados os cursos cuja diferença é maior que 50 pontos, considerando esse valor como um limite arbitrário para definir alta competitividade.
   
5. **Exibição dos Resultados:**
   - Finalmente, o código imprime no console a lista dos cursos considerados altamente competitivos juntamente com o valor da diferença.

In [ ]:
media_notas_candidatos = df.groupby('NO_CURSO')['NU_NOTA_CANDIDATO'].mean()

nota_corte = df.groupby('NO_CURSO')['NU_NOTACORTE_CONCORRIDA'].first()

diferenca = media_notas_candidatos - nota_corte

competitivos = diferenca[diferenca > 50]  # Definindo um limite arbitrário de 50 pontos

plt.figure(figsize=(10, 5))
sns.barplot(x=competitivos.index, y=competitivos.values, palette='viridis')
plt.xlabel("Cursos")
plt.ylabel("Diferença Média vs. Nota de Corte")
plt.title("Cursos Altamente Competitivos")
plt.xticks(rotation=90)
plt.show()

print("Cursos altamente competitivos:")
print(competitivos)

## Classificação média dos ingressantes por curso
Avaliar se há cursos onde os candidatos entram com maior ou menor vantagem na classificação.

1. **Cálculo da Classificação Média:**
   - O código agrupa o DataFrame pelo campo `NO_CURSO` e calcula a média dos valores na coluna `NU_CLASSIFICACAO` para cada curso.

2. **Exibição dos Resultados:**
   - O gráfico é exibido e, em seguida, o DataFrame com a classificação média por curso é impresso no console.

In [ ]:
media_classificacao = df.groupby('NO_CURSO')['NU_CLASSIFICACAO'].mean()

plt.figure(figsize=(10, 5))
sns.barplot(x=media_classificacao.index, y=media_classificacao.values, palette='coolwarm')
plt.xlabel("Cursos")
plt.ylabel("Classificação Média")
plt.title("Classificação Média dos Ingressantes por Curso")
plt.xticks(rotation=90)
plt.show()

print("\nClassificação média dos ingressantes por curso:")
print(media_classificacao)

media_df = pd.DataFrame({'Média Classificação': media_classificacao})
plt.figure(figsize=(10, max(5, len(media_df) * 0.5)))
sns.heatmap(media_df, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Heatmap - Classificação Média dos Ingressantes por Curso")
plt.show()

# 3. Desempenho Acadêmico e Perfil dos Ingressantes

## Correlação entre Tipo de Ensino Médio e Nota no ENEM
 Verificar se alunos de escolas públicas (ENSINO_MEDIO) têm notas médias (NU_NOTA_CANDIDATO) diferentes das de escolas privadas.

1. **Cálculo da Média das Notas do ENEM por Tipo de Ensino Médio:**
   - O código agrupa o DataFrame (`df`) pela coluna `ENSINO_MEDIO` e calcula a média das notas dos candidatos (`NU_NOTA_CANDIDATO`) para cada tipo de ensino médio.

2. **Exibição dos Resultados:**
   - O gráfico é exibido com `plt.show()`.
   - Em seguida, os valores médios calculados para cada tipo de ensino são impressos no console.

In [ ]:
enem_por_ensino = df.groupby('ENSINO_MEDIO')['NU_NOTA_CANDIDATO'].mean()

plt.figure(figsize=(8, 5))
sns.barplot(x=enem_por_ensino.index, y=enem_por_ensino.values, palette='Blues')
plt.xlabel("Tipo de Ensino Médio")
plt.ylabel("Média da Nota no ENEM")
plt.title("Correlação entre Tipo de Ensino Médio e Nota no ENEM")
plt.xticks(rotation=45)
plt.show()

print("\nCorrelação entre Tipo de Ensino Médio e Nota no ENEM:")
print(enem_por_ensino)

# 4. Distribuição Geográfica e Mobilidade

## Origem dos Estudantes
Cruzar a localização da universidade (SG_UF_IES) com o estado de origem dos candidatos (se disponível) para entender a mobilidade interestadual.

1. **Verificação de Coluna:**
   - O código primeiro verifica se a coluna `SG_UF_CANDIDATO` existe no DataFrame `df`. Se não existir, imprime uma mensagem informando a ausência da coluna.

2. **Cálculo da Mobilidade Interestadual:**
   - Quando a coluna existe, o código agrupa os dados por `SG_UF_CANDIDATO` (Estado de Origem do Candidato) e `SG_UF_IES` (Estado da Universidade) e conta o número de ocorrências para cada combinação.
   - O resultado é organizado em um formato de tabela (`mobilidade`) onde as linhas representam os estados de origem e as colunas os estados das universidades. Valores ausentes são preenchidos com zero usando `unstack(fill_value=0)`.

3. **Exibição dos Resultados:**
   - O heatmap é exibido na tela com `plt.show()`.
   - Por fim, a tabela `mobilidade` é impressa no console para visualização dos dados.

In [ ]:
if 'SG_UF_CANDIDATO' in df.columns:
    mobilidade = df.groupby(['SG_UF_CANDIDATO', 'SG_UF_IES']).size().unstack(fill_value=0)
    
    plt.figure(figsize=(12, 6))
    sns.heatmap(mobilidade, cmap='Blues', linewidths=0.5, annot=False)
    plt.xlabel("Estado da Universidade")
    plt.ylabel("Estado de Origem do Candidato")
    plt.title("Mobilidade Interestadual dos Estudantes")
    plt.show()
    
    print("Mobilidade Interestadual dos Estudantes:")
    print(mobilidade)
else:
    print("A coluna 'SG_UF_CANDIDATO' não está disponível nos dados.")

## Distribuição dos ingressantes entre campi da UFMT
Identificar quais campi atraem mais candidatos e se há variação nas notas de corte.

1. **Contagem de Ingressantes por Campus:**
   - É calculada a frequência de registros para cada campus usando `value_counts()` na coluna `NO_CAMPUS`.

2. **Exibição dos Campi com Mais Ingressantes:**
   - Imprime-se a contagem de ingressantes por campus no console.

3. **Cálculo de Média e Desvio Padrão das Notas de Corte:**
   - Agrupa os dados por campus e, para cada grupo, calcula a média e o desvio padrão das notas de corte.
   - Os resultados são impressos para exibir a distribuição dos valores estatísticos por campus.

In [ ]:
ingressantes_por_campus = df['NO_CAMPUS'].value_counts()

plt.figure(figsize=(12, 6))
sns.boxplot(x='NO_CAMPUS', y='NU_NOTACORTE_CONCORRIDA', data=df)
plt.xticks(rotation=90)
plt.xlabel("Campus")
plt.ylabel("Nota de Corte")
plt.title("Distribuição das Notas de Corte por Campus da UFMT")
plt.show()

print("Ingressantes por campus:")
print(ingressantes_por_campus)

notas_corte_por_campus = df.groupby('NO_CAMPUS')['NU_NOTACORTE_CONCORRIDA'].agg(['mean', 'std'])
print("\nMédia e desvio padrão das notas de corte por campus:")
print(notas_corte_por_campus)

# 5. Análise de Turnos e Tipos de Formação

## Preferência por Turno
Comparar a demanda por cursos no turno diurno vs. noturno (DS_TURNO). Há cursos com maior concorrência em um turno específico?

1. **Contagem da Demanda por Turno:**
   - O código utiliza `value_counts()` na coluna `DS_TURNO` para obter a quantidade de candidatos por turno.

2. **Exibição da Demanda por Turno:**
   - O gráfico é exibido e, em seguida, a contagem de candidatos por turno é impressa no console.

3. **Comparação da Concorrência por Turno:**
   - Os dados são agrupados pela coluna `DS_TURNO` e é calculada a média da concorrência (`QT_VAGAS_CONCORRENCIA`) para cada turno.
   - Esses valores médios são impressos no console para análise comparativa.

In [ ]:
demanda_por_turno = df['DS_TURNO'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=demanda_por_turno.index, y=demanda_por_turno.values, palette='viridis')
plt.xlabel("Turno")
plt.ylabel("Número de Candidatos")
plt.title("Demanda por Cursos no Turno Diurno vs. Noturno")
plt.show()

print("Demanda por turno:")
print(demanda_por_turno)

concorrencia_por_turno = df.groupby('DS_TURNO')['QT_VAGAS_CONCORRENCIA'].mean()
print("\nMédia de concorrência por turno:")
print(concorrencia_por_turno)


## Formação Acadêmica
Relacionar o tipo de formação (DS_FORMACAO – bacharelado, licenciatura) com a procura e notas de corte.

1. **Contagem de Candidatos por Formação Acadêmica:**
   - É utilizada a função `value_counts()` na coluna `DS_FORMACAO` para contar a quantidade de candidatos para cada tipo de formação acadêmica.

2. **Cálculo da Média das Notas de Corte por Formação:**
   - Os dados são agrupados pela coluna `DS_FORMACAO` e, para cada grupo, é calculada a média da nota de corte (`NU_NOTACORTE_CONCORRIDA`).
   - O resultado dessa média é impresso no console, permitindo analisar as notas de corte médias por tipo de formação.

In [ ]:
demanda_por_formacao = df['DS_FORMACAO'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=demanda_por_formacao.index, y=demanda_por_formacao.values, palette='magma')
plt.xlabel("Formação Acadêmica")
plt.ylabel("Número de Candidatos")
plt.title("Demanda por Tipo de Formação")
plt.show()

notas_corte_por_formacao = df.groupby('DS_FORMACAO')['NU_NOTACORTE_CONCORRIDA'].mean()
stats = pd.DataFrame({
    'Numero de Candidatos': demanda_por_formacao,
    'Nota de Corte Media': notas_corte_por_formacao
})

# Plotando o heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(stats, annot=True, cmap='magma', fmt=".1f")
plt.title("Heatmap - Demanda e Nota de Corte por Formação")
plt.show()

print("\nMédia das notas de corte por formação:")
print(notas_corte_por_formacao)

# 6. Comparação entre Modalidades de Concorrência


## Ampla Concorrência vs. Cotas
Analisar diferenças nas notas médias, classificação (NU_CLASSIFICACAO) e taxa de sucesso entre candidatos de ampla concorrência (TIPO_CONCORRENCIA) e aqueles em modalidades afirmativas.

1. **Criação da Coluna “COTA”:**
   - Uma nova coluna chamada `COTA` é criada a partir da coluna `TIPO_CONCORRENCIA` utilizando `apply` e uma função lambda.
   - Se o valor de `TIPO_CONCORRENCIA` for `AC`, é rotulado como "Ampla Concorrência"; do contrário, como "Cotas".

2. **Comparação de Notas Médias:**
   - O DataFrame é agrupado pela modalidade de concorrência (`COTA`) e é calculada a média das notas dos candidatos (`NU_NOTA_CANDIDATO`).
   - O resultado é impresso para comparar as notas médias entre as modalidades.

3. **Comparação de Classificação Média:**
   - Similarmente, é calculada a média das classificações (`NU_CLASSIFICACAO`) para cada modalidade de concorrência.
   - O resultado é exibido para analisar a variação nas classificações entre "Ampla Concorrência" e "Cotas".

4. **Cálculo da Taxa de Sucesso:**
   - É aplicada uma função utilizando `groupby` e `apply` para calcular a taxa de sucesso (aprovados) em cada modalidade.
   - Para cada grupo, a taxa de sucesso é definida como a fração de candidatos cuja classificação (`NU_CLASSIFICACAO`) é menor ou igual ao número de vagas concorridas (`QT_VAGAS_CONCORRENCIA`).

In [ ]:
df['COTA'] = df['TIPO_CONCORRENCIA'].apply(lambda x: 'Ampla Concorrência' if x == 'AC' else 'Cotas')

dif_notas = df.groupby('COTA')['NU_NOTA_CANDIDATO'].mean()
print("\nMédia das notas por modalidade de concorrência:")
print(dif_notas)

dif_classificacao = df.groupby('COTA')['NU_CLASSIFICACAO'].mean()
print("\nClassificação média por modalidade de concorrência:")
print(dif_classificacao)

taxa_sucesso = df.groupby('COTA').apply(lambda x: (x['NU_CLASSIFICACAO'] <= x['QT_VAGAS_CONCORRENCIA']).mean())
print("\nTaxa de sucesso por modalidade de concorrência:")
print(taxa_sucesso)

g = sns.FacetGrid(df, hue="COTA", height=6, aspect=1.5)
g.map(sns.kdeplot, "NU_NOTA_CANDIDATO", shade=True)
g.add_legend()
plt.title("Distribuição das Notas por Modalidade de Concorrência")
plt.xlabel("Nota do Candidato")
plt.ylabel("Densidade")
plt.show()